In [ ]:
#loading necessary libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import statsmodels.api as sm

In [ ]:
# Importing the datasets

train = pd.read_csv('train.csv', encoding = 'latin-1')
test = pd.read_csv('test.csv', encoding = 'latin-1')
store = pd.read_csv('store.csv', encoding = 'latin-1')

print("\n Train dataset dimensions: ", train.shape)
print("\n Test dataset dimensions: ", test.shape)
print("\n Store dataset dimensions: ", store.shape)

In [ ]:
train.head(10)

In [ ]:
test.head(10)

In [ ]:
store.head(10)

In [ ]:
train['Sales_per_customer'] = train['Sales']/train['Customers']
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
train[(train['Open'] == 0) & (train['Sales'] == 0)].shape
# i.e., of the 172869 records, 172817 records have empty sales and the store status is closed. 
# removing these records from the analysis would be beneficial to avoid data skewness - 10% of total records

# For the remaining 52 records, the sales numbers are still 0 
# Will be dropping those records as well.

In [ ]:
train[(train['Open'] != 0) & (train['Sales'] == 0)].shape
df = train.copy()

In [ ]:
train = train[(train['Open'] != 0) & (train['Sales'] != 0)]
train.shape

In [ ]:
train = train[train['Sales'] != 0]

print(np.mean(train.Sales))
print(np.median(train.Sales))

In [ ]:
store.isnull().sum()

In [ ]:
print(store.shape)

In [ ]:
# Replace competitor distance information with median value - outliers not sure
# Replace promo relevant information and competition open informations with 0

store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace = True)
store.fillna(0, inplace = True)

store.isnull().sum()

In [ ]:
# Joining both train and store information dataset

train_combined = pd.merge(train, store, how = 'inner', on = 'Store')
train_combined.shape

In [ ]:
# For ARIMA forecasting - considering a store from the dataset
arima_train = train_combined.copy()
arima_train = arima_train.set_index('Date')

arima_train.head(5)

In [ ]:
store85_sample = arima_train[arima_train.Store == 85].Sales
store85_sample = store85_sample.sort_index(axis = 0)
store85_sample.head(3)

In [ ]:
store85_sample.index = pd.to_datetime(store85_sample.index)
store85_sample.resample('MS').mean().plot(figsize = (25,10))
plt.show()

# from the plot, we observe some distibguishable patterns - like seasonality pattern i.e.,
# the sales are always low at the beginning of the year 
# tend to have a peak just before the year end
# and sales fall low at the year end

# Also, we obseve an upward trend within any single year around the mid of the year.

In [ ]:
store85_sample.index.min()
store85_sample.index.max()

In [ ]:
# Visualizing the data - 
# using time-series decomposition that allows us to decompose our time series data into 3 distinct components:
# Trend, Seasonality and noise

import statsmodels.api as sm
from pylab import rcParams

rcParams['figure.figsize'] = 20,12

decomposition = sm.tsa.seasonal_decompose(store85_sample, model = 'additive')
fig = decomposition.plot()
plt.show()


In [ ]:
# applying ARIMA
import itertools

p = range(10,12)
d = q = range(0,2)
pdq = list(itertools.product(p,d,q))

sp = sd = sq = range(0,3)
seasonal_pdq = [(x[0],x[1],x[2],12) for x in (list(itertools.product(sp,sd,sq)))]

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(store85_sample, order = param, 
                                           seasonal_order = param_seasonal,
                                           enforce_stationarity = False,
                                           enforce_invertibility = False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC : {}'.format(param, param_seasonal,results.aic))
        except:
            continue

In [ ]:
# From the above output lowet AIC value is for - 
# Considering SARMIAX(11,1,0)x(2,1,0,12)

model = sm.tsa.statespace.SARIMAX(store85_sample, order = (11,1,0), 
                                                  seasonal_order = (2,1,0,12),
                                                  enforce_stationarity = False,
                                                  enforce_invertibility = False)
results = model.fit()

print(results.aic)

print(results.summary().tables[1])

In [ ]:
# Model diagnostics - to investigate any unusual behavior. 
results.plot_diagnostics(figsize=(20,10))
plt.show()

# Results - not perfect but model residues are nearly normally distributed and same is suggested by the QQ plot

In [ ]:
# Validating forecasts:
from datetime import datetime
pred = results.get_prediction(start = pd.to_datetime('2015-01-01')) #, dynamic = 'False')
pred_ci = pred.conf_int()

ax = store85_sample['2013':].plot(label = 'observed')

pred.predicted_mean.plot(ax = ax, label = 'One-step ahead Forecast', alpha = 0.8, figsize = (20,8))

ax.fill_between(pred_ci.index, pred_ci.iloc[:,0], pred_ci.iloc[:,1], color = 'k', alpha = 0.2)

ax.set_xlabel('Date')
ax.set_ylabel('Store 85 Sales')

plt.legend()

plt.show()

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = store85_sample['2015-01-01':]

mse = ((y_forecasted - y_truth) ** 2).mean()

print('The Mean Squared Error is: {}'.format(round(mse,2)))

print('The Root Mean Squared Error is: {}'.format(round(np.sqrt(mse),2)))

In [ ]:
# Producing and visualizing forecasts 

pred_uc = results.get_forecast(steps = 100)
pred_ci = pred_uc.conf_int()

ax = store85_sample.plot(label = 'observed', figsize = (20,8))

pred_uc.predicted_mean.plot(ax = ax, label = 'Forecast')

ax.fill_between(pred_ci.index, pred_ci.iloc[:,0], pred_ci.iloc[:,1], color = 'k', alpha = 0.25)

ax.set_xlabel('Date')
ax.set_ylabel('Store 85 Sales')

plt.legend()

plt.show()

In [ ]:
# Time series forecasting using Prophet
# In 2017, Facebook released a forecasting tool Prophet designed for analyzing time series data that display patterns on 
# different time scales such as - yearly, weekly, daily
# It also has advanced capabilities for modeling the effect of holidats on time series and implementing custom changepoints.

from fbprophet import Prophet

store85_sample_df = train_combined[train_combined.Store == 85][['Date','Sales']]
store85_sample_df.rename(columns = {'Date': 'ds','Sales':'y'}, inplace = True)
store85_sample_df.head(5)

In [ ]:
prophet_model = Prophet(interval_width = 0.95)
prophet_model.fit(store85_sample_df)

forecast = prophet_model.make_future_dataframe(periods = 12, freq = 'MS')
forecast = prophet_model.predict(forecast)

plt.figure(figsize = (30,8))
prophet_model.plot(forecast, xlabel = 'Date', ylabel = 'Store 85 Sales')
plt.title('Store 85 Sales Forecast using Prophet')

In [ ]:
# Regression models - 
# Multiple Linear Regression, SVR, Decision tree, Random Forest, XGB
# Regularization methods - L1 , L2 regression